# <span style="color: green; font-size: 40px; font-weight: bold;"> Projeto  </span>

# Contexto

<br><br>

## Importando Pacotes

<br><br>

## Carregando Conjunto de Dados

<br><br><br>

# <span style="color: green; font-size: 34px; font-weight: bold;"> Análise Exploratória Inicial dos Dados </span>

<br>

### Função Para Análise Inicial

In [ ]:
# Função Genérica para uma Análise Inicial (para dataframes do tipo Pandas)

def analise_inicial(df):
    # Informações do DataFrame
    print('\n-------------------------------------------------------------------------------------------\n\n')
    print('INFO\n\n')
    df.info()

    # Verifica se há valores ausentes e duplicados
    valores_ausentes = df.isna().sum().sum() > 0
    valores_duplicados = df.duplicated().sum() > 0

    # Nomes das variáveis com valores ausentes
    variaveis_ausentes = df.columns[df.isna().any()].tolist()

    # Número de linhas duplicadas
    num_linhas_duplicadas = df.duplicated().sum()

    # Porcentagem de linhas duplicadas
    porcentagem_linhas_duplicadas = (num_linhas_duplicadas / len(df)) * 100

    # Total de linhas com pelo menos um valor ausente
    total_linhas_ausentes = df.isna().any(axis=1).sum()

    # Porcentagem de linhas com pelo menos um valor ausente
    porcentagem_linhas_ausentes = (total_linhas_ausentes / len(df)) * 100

    # Exibe o resultado
    if valores_ausentes:
        print("\n\nExistem valores ausentes:", valores_ausentes)
        print("\nVariáveis com valores ausentes:", variaveis_ausentes)
        print("\nTotal de Linhas com Valores Ausentes:", total_linhas_ausentes)
        print("\nPorcentagem de Linhas Com Valor Ausente: {:.2f}%".format(porcentagem_linhas_ausentes))
    else:
        print("\n\nNenhuma variável possui valores ausentes.")

    if valores_duplicados:
        print("\n\n\nExistem valores duplicados:", valores_duplicados)
        print("\nNúmero de Linhas Duplicadas:", num_linhas_duplicadas)
        print("\nPorcentagem de Linhas Duplicadas: {:.2f}%\n\n".format(porcentagem_linhas_duplicadas))
    else:
        print("\nNenhuma variável possui valores duplicados.\n\n")

analise_inicial(df_pandas)

In [ ]:
# Função Genérica para uma Análise Inicial (para dataframes do tipo PySpark)

def analise_inicial(df):
    # Exibir o schema do DataFrame
    print('\n\n INFO (schema)\n\n')
    df.printSchema()
    
    print('\n\n ------------------------------------------------------------------------------------------ \n\n')
    
    # Número de linhas
    print("Número de Linhas:", df.count())

    # Verificar valores ausentes em cada coluna
    print('\n\n TOTAL DE VALORES NaN por Coluna \n')
    valores_ausentes = df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns])
    valores_ausentes.show()

    # Verificar se existe alguma variável com valores ausentes
    valores_ausentes_boolean = any(row[c] > 0 for row in valores_ausentes.collect() for c in df.columns)

    # Nomes das variáveis com valores ausentes
    variaveis_ausentes = [c for c in df.columns if df.filter(col(c).isNull() | isnan(c)).count() > 0]

    # Número de linhas duplicadas
    num_linhas_total = df.count()
    num_linhas_distintas = df.distinct().count()
    num_linhas_duplicadas = num_linhas_total - num_linhas_distintas

    # Porcentagem de linhas duplicadas
    porcentagem_linhas_duplicadas = (num_linhas_duplicadas / num_linhas_total) * 100

    # Total de linhas com pelo menos um valor ausente
    total_linhas_ausentes = df.filter(
        " OR ".join([f"(`{c}` IS NULL OR isnan(`{c}`))" for c in df.columns])
    ).count()

    # Porcentagem de linhas com pelo menos um valor ausente
    porcentagem_linhas_ausentes = (total_linhas_ausentes / num_linhas_total) * 100

    # Exibe o resultado
    print("\n\nExistem valores ausentes:", valores_ausentes_boolean)
    
    if valores_ausentes_boolean:
        print("\nVariáveis com valores ausentes:", variaveis_ausentes)
        print("\nTotal de Linhas com Valores Ausentes:", total_linhas_ausentes)
        print("\nPorcentagem de Linhas Com Valor Ausente: {:.2f}%".format(porcentagem_linhas_ausentes))
    else:
        print("\nNenhuma variável possui valores ausentes.")

    print("\n\n\nExistem valores duplicados:", num_linhas_duplicadas > 0)
    
    if num_linhas_duplicadas > 0:
        print("\nNúmero de Linhas Duplicadas:", num_linhas_duplicadas)
        print("\nPorcentagem de Linhas Duplicadas: {:.2f}%\n".format(porcentagem_linhas_duplicadas))
    else:
        print("\nNenhuma variável possui valores duplicados.\n")

# Exemplo de uso da função com o DataFrame df_spark
analise_inicial(df_spark)

### Resumo

- 

<br><br>

# Visualizando os Tipos dos Dados Separadamente

<br>

#### Visualizando Variáveis Categóricas e Numéricas

In [ ]:
# Exibindo Variáveis Categóricas (filtrando)
dados.dtypes[dados.dtypes == 'object']

In [ ]:
# Exibindo Variáveis Numéricas (filtrando)
dados.dtypes[dados.dtypes != 'object']

<br><br>

## Analisando Variáveis Categóricas

#### Resumo Estatístico

In [ ]:
# Describe (informando que é para somente variáveis categóricas)
print('\nDescribe\n')
display(dados.describe(include = ['object']))
print('\n------------------------------------------------------------------------\n\n')

# Verificando Tipo das Variáveis (adicionar mais variáveis se necessário)
print('\nTipo das Variáveis\n')
print(dados['genre'].unique())

In [ ]:
# Exibir os gráficos de contagem para todas as variáveis categóricas (adicionar variáveis categóricas necessárias)
categorical_vars = ['genre']

for var in categorical_vars:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=dados, x=var, label='Count')
    plt.title(f'Contagem de {var}')
    plt.xlabel(var.replace('_', ' ').capitalize())
    plt.ylabel('Contagem')
    plt.xticks(rotation=45)
    plt.show()

# Value counts para todas as variáveis categóricas
for var in categorical_vars:
    var_counts = dados[var].value_counts()
    print(f"\nContagem de {var.replace('_', ' ').capitalize()}:")
    for category, count in var_counts.items():
        print(f'{category}: {count}')

### Resumo da Análise

- 

<br>

## Aplicando Label Encoding

In [ ]:
# Lista de variáveis categóricas (adicionar variáveis categóricas necessárias)
categorical_vars = ['genre']

# Aplicando Label Encoding
label_encoders = {}
for var in categorical_vars:
    le = LabelEncoder()
    dados[var] = le.fit_transform(dados[var])
    label_encoders[var] = le
    
# Verificando os tipos das variáveis
print('\nTipos das Variáveis Após Label Encoding\n')
print(dados.dtypes)

# Verificando os valores únicos de 'income'
print('\n\n-------------------------------------------------------------------------------------------------')
print('\nValores Únicos da Variável Alvo (genre)\n')
print(dados['genre'].value_counts())

<br><br>

## Analisando Todas as Variáveis

#### Resumo Estatístico

In [ ]:
# Salvar o formato original
original_float_format = pd.options.display.float_format

# Ajustar a exibição do pandas para valores sem notação científica
pd.options.display.float_format = '{:.2f}'.format

# Verificando o resumo estatístico sem notação científica
print('\nSem Notação Científica')
display(dados.describe())
print('\n----------------------------------------------------------------------------------------------\n\n')

# Restaurar o formato original
pd.options.display.float_format = original_float_format

# Verificando o resumo estatístico novamente para confirmar que voltou ao normal
#print('\nCom Notação Científica')
#display(dados.describe())

<br>

#### Visualizando através de Gráficos Histograma

In [ ]:
# Plot
dados.hist(figsize = (15,15), bins = 10) 
plt.show()

print('\n\n------------------------------------------------------------------------------------------------\n')

# Visualização dos outliers
plt.figure(figsize=(15, 10))
sns.boxplot(data=dados)
plt.xticks(rotation=90)
plt.title('Boxplot para Detecção de Outliers')
plt.show()

### Resumo da Análise

<table border="2">
  <tr>
    <th style="text-align: center; font-size: 16px;">Nome Da Varivel</th>
    <th style="text-align: center; font-size: 16px;">Resumo_Análise</th>
    <th style="text-align: center; font-size: 16px;">Tratamento Antes Divisão</th>
    <th style="text-align: center; font-size: 16px;">Tratamento Depois Divisão</th>
  </tr>
  <tr>
    <td>energy</td>
    <td>Distribuição assimétrica com muitos valores altos, concentrando-se entre 0.6 e 1.0.</td>
    <td>Remoção de Outliers</td>
    <td>Transformação para Normalização</td>
  </tr>
  <tr>
    <td>speechiness</td>
    <td>Distribuição assimétrica, com muitos valores baixos concentrando-se perto de 0.</td>
    <td>Remoção de Outliers</td>
    <td>Transformação para Normalização</td>
  </tr>
  <tr>
    <td>genre</td>
    <td>Variável Alvo. Distribuição categórica dos gêneros musicais, já codificada numericamente.</td>
    <td>Não Necessita de Tratamento</td>
    <td>Não Necessita de Tratamento</td>
  </tr>
</table>

<br>

# Limpeza nos Dados

<br>

### Tratando Valores Ausentes

<br>

### Tratando Valores Duplicados

<br>

### Tratando Valores Outliers

- Apenas Remoção de Linhas

<br><br>

# Features Engineering (se necessário)

<br>

# Clusterização

- A técnica de **clusterização cria uma nova variável** que representa os clusters identificados no conjunto de dados. Esta nova variável pode ser usada como uma feature adicional na criação de modelos preditivos, fornecendo informações sobre os padrões identificados durante a clusterização.
- Verificar **variáveis relevantes** para a clusterização.
- Verificar no gráfico de cotovelo 

<br>

#### Preparação dos Dados

In [ ]:
## Preparação dos Dados

# Selecionar as variáveis relevantes para a clusterização (verificar veriáveis)
variables = ['']

# Criar um DataFrame apenas com as variáveis selecionadas
cluster_data = dados[variables]

# Normalizar os dados
scaler = StandardScaler()
cluster_data_scaled = scaler.fit_transform(cluster_data)


## Escolha do Algoritmo de Clusterização e Avaliação do Número Ótimo de Clusters

# Avaliar o método do cotovelo para encontrar o número ótimo de clusters
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(cluster_data_scaled)
    sse.append(kmeans.inertia_)

# Plotar o gráfico do método do cotovelo
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), sse, marker='o')
plt.title('Método do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('SSE')
plt.show()

#### Execução da Clusterização

In [ ]:
## Execução da Clusterização e Criação da Nova Variável

# Escolher o número de clusters com base no gráfico do método do cotovelo (onde a cauda começa a abaixar)
optimal_clusters = 4

# Aplicar K-Means com o número ótimo de clusters
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
dados['Cluster'] = kmeans.fit_predict(cluster_data_scaled)

# Verificar as novas colunas criadas
print("\nNovas colunas criadas:")
print(dados.columns)

# Exibir uma amostra do DataFrame para verificar as novas colunas
display(dados.head(10))

# Visualizar os clusters resultantes
plt.figure(figsize=(15, 10))
sns.scatterplot(x='Total_Damages', y='Total_Affected', hue='Cluster', data=dados, palette='viridis')
plt.title('Clusterização de Regiões e Tipos de Desastres')
plt.xlabel('Total de Danos')
plt.ylabel('Total de Afetados')
plt.legend(title='Cluster')
plt.show()

# Mostrar a contagem de observações em cada cluster
print(dados['Cluster'].value_counts())

# Analisar as características médias de cada cluster
cluster_summary = dados.groupby('Cluster')[variables].mean()
display(cluster_summary)

#### Verificando Desempenho da Clusterização

In [ ]:
from sklearn.metrics import silhouette_score

# Calcular a pontuação de silhueta
silhouette_avg = silhouette_score(cluster_data_scaled, dados['Cluster'])
print(f'Pontuação de Silhueta: {silhouette_avg:.2f}')

<br><br>

# Verificando Correlação

In [ ]:
## Verificando Correlações através de Tabela

# Criando Tabela
display(dados.corr())

print('\n\n=================================================================================================\n')


## Visualizando Correlações através de um Mapa de Calor

# Criando o Heatmap
corr_matrix = dados.select_dtypes(include=[np.number]).corr()

plt.figure(figsize=(16, 12))  # Define o tamanho da figura maior
heatmap = sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1, cbar=True, square=True, annot_kws={"size": 10})
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=12)  # Aumenta a fonte das labels
heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=0, fontsize=12)  # Aumenta a fonte das labels
plt.title('Mapa de Calor das Correlações', fontsize=18)  # Aumenta o título
plt.tight_layout()  # Ajusta o layout para evitar corte de labels
plt.show()


print('\n\n---------------------------------------------------------------------------------------------------\n')

# Calcular correlação com a variável alvo
correlation_target = dados_ml.corr()['ganhos_mensais_estimados_minimo'].drop('ganhos_mensais_estimados_minimo')

# Ordenar pela correlação absoluta
correlation_target_sorted = correlation_target.abs().sort_values(ascending=False)

# Visualizar as correlações
plt.figure(figsize=(12, 8))
sns.barplot(x=correlation_target_sorted.values, y=correlation_target_sorted.index, palette='coolwarm')
plt.title('Correlação com Ganhos Mensais Estimados Mínimos')
plt.xlabel('Correlação')
plt.show()

# Exibir correlações
display(correlation_target_sorted)

<br>

## Resumo da Análise:

#### Análise das Principais Correlações com a Variável Alvo

- 

#### Variáveis a serem removidas:
- 

<br><br>

# Seleção de Variáveis Usando RandomForest

<br>

### Conclusão

- Combinando a **análise de correlação** e a **análise de importância das variáveis**, a recomendação é

<br> <br> 

# Engenharia de Atributos

- Aplicar técnica de VectorAssembler em objetos PySpark

<br>

### Aplicando Função VectorAssembler 

- Criando um **Objeto Vetor de Atributos** e adicionando ao Dataframe
- As **colunas selecionadas** foram escolhidas a partir das análises feita anteriormente.

In [ ]:
# Prepara o vetor de atributos (Este processo combina as colunas definidas em uma única coluna vetorial)
assembler = VectorAssembler(inputCols = ['Open', 'VolBTC', 'VolCurrency'], 
                            outputCol = "features")

# Adicionar a nova coluna ao DataFrame
df_assembled = assembler.transform(dados)

# Verificando os Tipos das Colunas
df_assembled.printSchema()

<br><br><br><br>

# Pré-Processamento de Dados Para Construção de Modelos de Machine Learning

<br><br>


## Dividindo os dados em Dados de Treino e Dados de Teste
- Nós **treinamos** o modelo com **dados de treino** e **avaliamos** o modelo com **dados de teste**.

<br>

In [ ]:
# Cria um objeto separado para a variável alvo
y = dados.income

# Cria um objeto separado para as variáveis de entrada
X = dados.drop('income', axis=1)

# Split em dados de treino e teste sem amostragem estratificada
X_treino, X_teste, y_treino, y_teste = train_test_split(X, 
                                                        y, 
                                                        test_size=0.2, 
                                                        random_state=1234)

# Print do shape
print(X_treino.shape, X_teste.shape, y_treino.shape, y_teste.shape)

<br>

### Tratando Valores Outliers

<br>

> **Importante**:
Não devemos modificar a escala antes da divisão em treino e teste para evitar vazamento de dados, pois isso garantiria que informações dos dados de teste influenciem a transformação, o que poderia levar a um modelo excessivamente otimista e menos generalizável para dados não vistos.

> **Importante 2**:
A abordagem correta envolve ajustar o Tips de Tratamento de Outliers e o escalonamento nos dados de treino e aplicar esses mesmos parâmetros aos dados de teste para manter a consistência.

> **Sugestão**:
É mais apropriado tratar cada variável **individualmente**, pois cada uma possui uma distribuição e comportamento diferente. Tratando-as individualmente, podemos aplicar métodos específicos para cada caso, o que resulta em uma limpeza mais precisa e adequada para o modelo.

<br>

### Tipos de Tratamento de Outliers

<br>

#### Técnicas Padrão:

> **Winsorização**: Limita os valores dos outliers a limites percentuais definidos, substituindo valores extremos.

- **Vantagens**: Preserva todos os dados, apenas ajustando valores extremos e fácil de implementar e entender.
- **Desvantagens**: Pode distorcer a distribuição dos dados e os limites precisam ser escolhidos com cuidado.

<br>

> **Remoção de Linhas**: Remove registros contendo outliers. Pode resultar na perda de dados valiosos, mas é útil quando os outliers são extremamente discrepantes.

- **Vantagens**: Eficaz quando os outliers são poucos e claramente discrepantes.
- **Desvantagens**: Pode levar à perda significativa de dados e não aplicável se muitos outliers estiverem presentes.

<br>

> **Capping (Truncation)**: Limita os valores dos dados a um máximo e um mínimo definidos, substituindo os outliers pelos limites.

- **Vantagens**: Mantém todos os dados, evitando a remoção de registros.
- **Desvantagens**: Pode introduzir viés ao fixar valores nos limites e os limites precisam ser escolhidos com cuidado.

<br>

#### Técnicas Avançadas:

> **Detecção de Outliers com Isolation Forest**: Algoritmo de aprendizado não supervisionado que identifica e trata outliers com base em padrões aprendidos nos dados.

- **Vantagens**: Eficaz para grandes conjuntos de dados e alta dimensionalidade e não precisa de distribuição normal dos dados
- **Desvantagens**: Pode ser complexo de implementar e parâmetros podem ser difíceis de ajustar.

> **Detecção de Outliers com Local Outlier Factor (LOF)**: Algoritmo de aprendizado não supervisionado que identifica outliers com base na densidade local dos dados.

- **Vantagens**: Detecta outliers em regiões de diferentes densidades.
- **Desvantagens**: Pode ser computacionalmente intensivo e sensível ao número de vizinhos (parâmetro k).

<br>

> **Clusterização**: Utiliza algoritmos de clusterização para identificar e tratar outliers como pontos em clusters diferentes ou distantes.

- **Vantagens**: Pode identificar outliers como pontos em clusters menores ou distantes e é adaptável a diferentes formas de dados.
- **Desvantagens**: Requer escolha de número de clusters e computacionalmente caro.

<br>

In [ ]:
## Tratando Valores Outliers

# Função para aplicar Winsorização
def apply_winsorization(X, limits):
    return X.apply(lambda col: winsorize(col, limits=limits))

# Limites para Winsorização
winsor_limits = (0.05, 0.05)

# Obter as colunas de X_treino exceto a(s) indicada(s)
variables_to_winsorize = X_treino.columns.difference(['idade_do_canal'])

# Winsorização das variáveis numéricas contínuas nos dados de treino e teste
X_treino[variables_to_winsorize] = apply_winsorization(X_treino[variables_to_winsorize], winsor_limits)
X_teste[variables_to_winsorize] = apply_winsorization(X_teste[variables_to_winsorize], winsor_limits)

# Winsorização da variável alvo de treino e teste
y_treino = winsorize(y_treino, winsor_limits)
y_teste = winsorize(y_teste, winsor_limits)

<br>

### Tratamento de Escala

> **Normalização** ou **Padronização**

- A transformação (como escalonamento) precisa ser aplicada nos dados de teste usando os parâmetros calculados a partir dos dados de treino para garantir a consistência. 

<br>

#### Padronizando

In [47]:
## Padronização

# Separar as colunas que não serão padronizadas
non_scaled_columns = ['idade_do_canal']
X_treino_non_scaled = X_treino[non_scaled_columns]
X_teste_non_scaled = X_teste[non_scaled_columns]

X_treino_to_scale = X_treino.drop(columns=non_scaled_columns)
X_teste_to_scale = X_teste.drop(columns=non_scaled_columns)

# Padronização dos dados de treino e teste
scaler_X = StandardScaler()
X_treino_scaled = scaler_X.fit_transform(X_treino_to_scale)
X_teste_scaled = scaler_X.transform(X_teste_to_scale)

# Reconstruir os DataFrames com as colunas não padronizadas
X_treino_scaled = pd.DataFrame(X_treino_scaled, columns=X_treino_to_scale.columns, index=X_treino.index)
X_teste_scaled = pd.DataFrame(X_teste_scaled, columns=X_teste_to_scale.columns, index=X_teste.index)

# Manter a coluna 'idade_do_canal' na mesma posição
X_treino = pd.concat([X_treino_scaled, X_treino_non_scaled], axis=1)
X_treino = X_treino[X.columns]
X_teste = pd.concat([X_teste_scaled, X_teste_non_scaled], axis=1)
X_teste = X_teste[X.columns]

# Padronização da variável alvo de treino e teste
scaler_y = StandardScaler()
y_treino_scaled = scaler_y.fit_transform(y_treino.reshape(-1, 1))
y_teste_scaled = scaler_y.transform(y_teste.reshape(-1, 1))

# Suponha que este é o DataFrame de treino original antes do ajuste
X_treino_columns = X_treino.columns
print("Colunas após tratamento:", X_treino_columns)

(1031, 14) (258, 14) (1031,) (258,)


<br><br><br><br><br>

<span style="color: green; font-size: 40px; font-weight: bold;">Construindo Modelos de Machine Learning</span>

<br>

- Nesta etapa do projeto o ideal é escolher um algoritmo simples e fácil de compreender, que será usado como Benchmark (modelo base).

#### Importante

- Iremos treinar dois conjuntos de dados: um **conjunto de dados com todas as variáveis** e outro **variáveis selecionadas**.

<br><br><br>

## Criando Dataframe para salvar métricas de cada Modelo

In [ ]:
# Cria um dataframe para receber as métricas de cada modelo
df_modelos = pd.DataFrame()

<br><br><br>

# <span style="color: green; font-weight: bold;">Modelo 1 com Regressão Logística (Benchmark)</span>

<br>

> # Versão 1

- Sem Ajuste de Hiperparâmetros


<br>

### Criação, Treinamento, Previsão e Avaliação do Modelo

<br>

### Salvando as Métricas

In [ ]:
# Criando um DataFrame para salvar as métricas
modelo_v1 = pd.DataFrame({
    'Nome do Modelo': ['Logistic Regression	'],
    'Versao': ['1'],
    'Tipo de Dados': ['Todas as Variáveis'],
    'Tipo de Modelo': ['Sem Ajuste de Hiperparâmetros'],
    'MAE': [f"{mae:.2f}"],
    'MSE': [f"{mse:.2f}"],
    'RMSE': [rmse],
    'Coeficiente R2': [r2],
    'Variância Explicada': [evs]
})

# Concatenando com o DataFrame existente
df_modelos = pd.concat([df_modelos, modelo_v1], ignore_index=True)

# Visualizando DataFrame
display(df_modelos)

<br>

> # Versão 2

- Com Ajuste de Hiperparâmetros

<br>


### Configurando Hiperparâmetros

<br>

### Criação, Treinamento, Previsão e Avaliação do Modelo

<br>

### Salvando as Métricas

<br><br>

# SELECIONANDO O MELHOR MODELO

- Usaremos o modelo que .

<br>

#### Visualizando Dataframe Ordenado

In [ ]:
# Ordenando o DataFrame pelo (modificar sort_values)
df_modelos_sorted = df_modelos.sort_values(by='AUC Score', ascending=False).reset_index(drop=True)

# Visualizando Daframe
display(df_modelos_sorted)

<br><br>

## Salvando e Carregando Melhor Modelo

<br><br>

# Salvando Pipeline com Melhor Modelo e Escalas

In [64]:
# Selecionar o melhor modelo com base no RMSE
melhor_modelo = df_modelos.nsmallest(1, 'RMSE').iloc[0]

# Visualizando dados do melhor modelo
display(melhor_modelo)

# Salvar os parâmetros das transformações e o melhor modelo
joblib.dump(scaler_X, 'pipeline_projeto_youtube/scaler_X.pkl')
joblib.dump(scaler_y, 'pipeline_projeto_youtube/scaler_y.pkl')
joblib.dump(modelo_GBR_v2, 'pipeline_projeto_youtube/modelo_XGB_v2.pkl')
joblib.dump(winsor_limits, 'pipeline_projeto_youtube/winsor_limits.pkl')

joblib.dump(label_encoders, 'pipeline_projeto_youtube/label_encoders.pkl') # Salvar todos os LabelEncoders

joblib.dump(X_treino_columns, 'pipeline_projeto_youtube/X_treino_columns.pkl') # Salvar os nomes das colunas


print('Arquivos salvos com sucesso.')

Nome do Modelo                     XGBoost Regressor
Versao                                             2
Tipo de Dados                              Reduzidos
Tipo de Modelo         Com Ajuste de Hiperparâmetros
MAE                                       686.908511
MSE                                   8098442.845715
RMSE                                     2845.776317
Coeficiente R2                              0.994235
Variância Explicada                         0.994277
Name: 11, dtype: object

Arquivos salvos com sucesso.


<br><br>

# Carregando Modelo e Pipeline Salvos

<br><br>

# Previsões com Novos Dados

<br><br><br><br>

# <span style="color: green; font-size: 38px; font-weight: bold;">CRIANDO INTERFACE</span>

<br>

# Versão 1

- Usando **Jupyter Widgets**.

<br><br>

====================================================================================================================

<br><br>

# Versão 2

- Versão mais elaborada usando o **Streamlit**.

<table border="2" style="font-size: 14px; border-spacing: 10px;">
  <caption style="font-size: 32px; margin: 30px; text-align: center">Pipeline para Modelos de Machine Learning (ML)</caption>
  <tr>
    <th style="text-align: center; font-size: 20px;">Passo</th>
    <th style="text-align: center; font-size: 20px;">Descrição</th>
    <th style="text-align: center; font-size: 20px;">Comentário</th>
  </tr>
  <tr>
    <td>1</td>
    <td>Definir Objetivo e Pergunta de Negócio</td>
    <td>Determinar o objetivo do projeto e as principais perguntas de negócio a serem respondidas.</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Importar Pacotes</td>
    <td>Importar as bibliotecas e pacotes necessários para o projeto.</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Carregar dados</td>
    <td>Carregar os dados é a primeira etapa essencial para qualquer projeto de Data Science.</td>
  </tr>
  <tr>
    <td>4</td>
    <td>Analisar Dados de forma Geral</td>
    <td>Realizar uma análise exploratória inicial para entender a estrutura e o conteúdo dos dados.</td>
  </tr>
  <tr>
    <td>5</td>
    <td>Remover Variáveis Completamente Irrelevantes</td>
    <td>Remover variáveis que não contribuem para a análise ou que são completamente irrelevantes.</td>
  </tr>
  <tr>
    <td>6</td>
    <td>Renomear Variáveis (se necessário)</td>
    <td>Renomear as variáveis para melhor visualização e entendimento dos dados.</td>
  </tr>
  <tr>
    <td>7</td>
    <td>Analisar Variáveis Categóricas</td>
    <td>Entender a distribuição e as características das variáveis categóricas.</td>
  </tr>
  <tr>
    <td>8</td>
    <td>Aplicar Tratamento nas Variáveis Categóricas antes do Label Encode (se necessário)</td>
    <td>Tratamentos como combinação de categorias raras ou correção de inconsistências devem ser feitos antes da codificação.</td>
  </tr>
  <tr>
    <td>9</td>
    <td>Aplicar Label Encode nas Variáveis Categóricas</td>
    <td>Converter variáveis categóricas em numéricas.</td>
  </tr>
  <tr>
    <td>10</td>
    <td>Analisar Todas as Variáveis com Describe e Gráficos</td>
    <td>Realizar uma análise estatística e visual de todas as variáveis para identificar padrões e anomalias.</td>
  </tr>
  <tr>
    <td>11</td>
    <td>Tratar Valores Ausentes</td>
    <td>Tratar valores ausentes antes da modelagem para evitar problemas.</td>
  </tr>
  <tr>
    <td>12</td>
    <td>Tratar Linhas Duplicadas</td>
    <td>Remover ou lidar com linhas duplicadas para garantir a qualidade dos dados.</td>
  </tr>
  <tr>
    <td>13</td>
    <td>Tratar Valores Outliers</td>
    <td>Identificar e tratar outliers que possam afetar a performance do modelo.</td>
  </tr>
  <tr>
    <td>14</td>
    <td>Feature Engineering (se necessário)</td>
    <td>Criar novas variáveis ou transformar variáveis existentes para melhorar a capacidade preditiva do modelo.</td>
  </tr>
  <tr>
    <td>15</td>
    <td>Aplicar Clusterização (se necessário)</td>
    <td>Identificar padrões ou segmentos nos dados para insights adicionais.</td>
  </tr>
  <tr>
    <td>16</td>
    <td>Análise de Correlação</td>
    <td>Identificar relações entre as variáveis que podem informar a seleção de variáveis.</td>
  </tr>
  <tr>
    <td>17</td>
    <td>Seleção de Variáveis</td>
    <td>Selecionar as variáveis mais relevantes para a modelagem.</td>
  </tr>
  <tr>
    <td>18</td>
    <td>Remover Variáveis Após Análise de Correlação e Seleção de Variáveis (se necessário)</td>
    <td>Remover variáveis que não são relevantes com base nas análises de correlação e seleção de variáveis.</td>
  </tr>  
  <tr>
    <td>19</td>
    <td>Dividir os Dados em Treino e Teste</td>
    <td>Separar os dados para validar o desempenho do modelo.</td>
  </tr>
  <tr>
    <td>20</td>
    <td>Tratar Outliers que Envolvam Mudança de Escala</td>
    <td>Aplicar tratamentos como winsorização e mudanças de escala.</td>
  </tr>  
  <tr>
    <td>21</td>
    <td>Aplicar Normalização ou Padronização (Depende dos Dados)</td>
    <td>Normalizar ou padronizar os dados após a divisão em treino e teste para evitar vazamento de dados.</td>
  </tr>
  <tr>
    <td>22</td>
    <td>Criar Modelo Benchmark</td>
    <td>Criar um modelo inicial simples para ter uma linha de base de desempenho.</td>
  </tr>
  <tr>
    <td>23</td>
    <td>Melhorar o Conjunto de Dados (se necessário)</td>
    <td>Melhorar os dados, se necessário, para tentar aumentar o desempenho do modelo.</td>
  </tr>
  <tr>
    <td>24</td>
    <td>Treinar Outros Algoritmos</td>
    <td>Testar múltiplos algoritmos para encontrar o mais adequado.</td>
  </tr>
  <tr>
    <td>25</td>
    <td>Verificar Desempenho dos Algoritmos</td>
    <td>Comparar os desempenhos para selecionar o melhor modelo.</td>
  </tr>
  <tr>
    <td>26</td>
    <td>Escolher o Melhor Modelo</td>
    <td>Selecionar o modelo com melhor desempenho.</td>
  </tr>
  <tr>
    <td>27</td>
    <td>Salvar Pipeline com Informações sobre o Projeto</td>
    <td>Selecionar o modelo com melhor desempenho e salvá-lo para uso futuro, incluindo no pipeline as informações necessárias para desnormalizar ou despadronizar os dados, bem como os labels e outras configurações.</td>
  </tr>
  <tr>
    <td>28</td>
    <td>Testar o Modelo com Dados Novos</td>
    <td>Validar o modelo com novos dados para verificar sua capacidade de generalização.</td>
  </tr>
  <tr>
    <td>29</td>
    <td>Criar uma Interface Gráfica do Modelo</td>
    <td>Criar uma interface gráfica pode ser útil, mas não é um passo essencial em todos os projetos de Data Science.</td>
  </tr>
</table>

<br>

# Padronização x Normalização

As técnicas de padronização e normalização são usadas no pré-processamento de dados em aprendizado de máquina para preparar variáveis numéricas, ajustando suas escalas. Aqui está quando e por que usar cada uma:

<br>

### Padronização
Transforma os dados de modo que eles tenham média zero e desvio padrão igual a um. 
- **Quando usar**: Aplicável quando os dados já estão centralizados em torno de uma média e precisam de ajuste na escala. É útil em modelos como SVM e Regressão Logística, que são sensíveis a variações na escala das variáveis de entrada.
- **Exemplo prático**: Se medimos altura em centímetros (150-190 cm) e peso em quilogramas (50-100 kg), a padronização permite comparar essas medidas numa escala comum, evitando distorções devido a diferentes intervalos de valores.
- **Por que escolher para este projeto**: Optamos pela padronização porque as variáveis têm escalas muito diferentes e há a presença de outliers significativos. A padronização mantém as propriedades estatísticas dos dados, minimizando o impacto dos outliers, ao contrário da normalização que pode distorcer os dados ao comprimir a maioria dos valores em um intervalo estreito.

<br>

### Normalização
Ajusta os dados para que seus valores caibam em um intervalo predefinido, geralmente de **0 a 1**.
- **Quando usar**: Ideal para dados com variações extremas nas escalas e onde os algoritmos são sensíveis à magnitude absoluta dos dados, como K-Nearest Neighbors (KNN) e técnicas de clustering.
- **Exemplo prático**: Se um dataset contém preços de produtos variando de R$1 a R$1000 e quantidades vendidas de 1 a 20 unidades, a normalização faria com que ambos os atributos tivessem a mesma contribuição no modelo, independentemente da escala original.
- **Por que não usamos aqui**: Não foi escolhida devido à presença de outliers, que poderiam ser enfatizados indevidamente, e porque a normalização poderia limitar a eficácia de modelos que assumem uma distribuição normal dos dados.

<br>

#### Importante:
- **Não é necessário** aplicar padronização/normalização na **variável alvo**.
- Nós **não aplicamos** as duas técnicas, ou usamos uma ou outra.
- A **normalização** pode não ser a melhor escolha se houver **outliers significativos no conjunto de dados**, pois isso poderia comprimir a maioria dos dados em um intervalo muito estreito. Nesses casos, a **padronização é recomendada**.

<br>

<br> <br>

# Análise Temporal

## Informações Necessárias para Análise Temporal

### Para fazer uma análise temporal, é obrigatório ter:

- **Variável Alvo (Target Variable)**: A variável que você deseja prever, como ganhos_mensais_estimados_minimo.
- **Marca Temporal (Timestamp)**: Uma coluna que representa o tempo em que cada observação foi feita. Isso pode ser uma data, hora ou uma combinação de ambas. Pode ser derivado de novas colunas criadas a partir de uma coluna de Data como dia_criacao_do_canal, mes_criacao_do_canal e ano_criacao_do_canal.

### Qual formato os Dados devem estar ?

Os dados devem estar no formato adequado:

- **Marca Temporal (Timestamp)**: Deve estar no tipo de dados datetime. No pandas, você pode usar pd.to_datetime para converter uma coluna para o tipo datetime.
- **Variável Alvo e Outras Variáveis**: Normalmente devem ser numéricas (int, float).

### Divisão do Conjunto de Dados em Treino e Teste

> É uma **boa prática** dividir os dados em conjuntos de treino e teste, mesmo para análises temporais. Isso permite avaliar o desempenho do modelo de previsão. No entanto, a divisão deve respeitar a sequência temporal:

- **Treino**: Usar dados históricos até um certo ponto no tempo.
- **Teste**: Usar dados subsequentes para avaliar a previsão.

### Avaliação do Modelo

> Existem várias métricas para avaliar modelos de séries temporais:

- **MAE (Mean Absolute Error)**: Média dos erros absolutos.
- **MSE (Mean Squared Error)**: Média dos erros quadrados.
- **RMSE (Root Mean Squared Error)**: Raiz quadrada da média dos erros quadrados.
- **MAPE (Mean Absolute Percentage Error)**: Média dos erros absolutos percentuais.

<br> <br>

# Exemplo de Preparação de Dados e Criação de Modelo com Regressão

In [53]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib

# Exemplo de dados
np.random.seed(42)  # Para reprodutibilidade

# Gerar dados
N = 1000
feature1 = np.random.randn(N) * 100 + 500
feature2 = np.random.randn(N) * 100 + 1500
target = 3 * feature1 + 2 * feature2 + np.random.randn(N) * 50  # Combinação linear com ruído

# Criar DataFrame
dados = pd.DataFrame({
    'feature1': feature1,
    'feature2': feature2,
    'target': target
})

# Exibir os primeiros dados
dados.head()

,feature1,feature2,target
0,549.671415,1639.935544,4895.126419
1,486.173570,1592.463368,4636.221513
2,564.768854,1505.963037,4666.611639
3,652.302986,1435.306322,4812.123525
4,476.584663,1569.822331,4474.717917


In [54]:
## Separar os dados em treino e teste

# Cria um objeto separado para a variável alvo
y = dados['target']

# Cria um objeto separado para as variáveis de entrada
X = dados.drop('target', axis=1)

# Split em dados de treino e teste sem amostragem estratificada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print do shape
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(800, 2) (200, 2) (800,) (200,)


In [55]:
## Tratando Valores Outliers

# Função para aplicar Winsorização
def apply_winsorization(X, limits):
    return X.apply(lambda col: winsorize(col, limits=limits))

winsor_limits = (0.05, 0.05)

# Winsorização dos dados de treino e teste
X_train_winsor = apply_winsorization(X_train, winsor_limits)
X_test_winsor = apply_winsorization(X_test, winsor_limits)

# Winsorização da variável alvo de treino e teste
y_train_winsor = winsorize(y_train, winsor_limits)
y_test_winsor = winsorize(y_test, winsor_limits)

In [56]:
## Padronização

# Padronização dos dados de treino e teste
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train_winsor)
X_test_scaled = scaler_X.transform(X_test_winsor)

# Padronização da variável alvo de treino e teste
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train_winsor.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test_winsor.reshape(-1, 1))

In [57]:
## Treinando o Modelo

# Treinar o modelo de regressão linear
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)


## Previsões

# Fazer previsões nos dados de teste
y_pred_scaled = model.predict(X_test_scaled)

# Reverter a padronização das previsões e dos valores reais para a escala original
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test_scaled)


## Avaliação

# Calcular métricas de avaliação
r2 = r2_score(y_test_original, y_pred)
mae = mean_absolute_error(y_test_original, y_pred)
mse = mean_squared_error(y_test_original, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred))
evs = explained_variance_score(y_test_original, y_pred)

print('MAE - Erro Médio Absoluto:', mae)
print('MSE - Erro Quadrático Médio:', mse)
print('RMSE - Raiz Quadrada do Erro Quadrático Médio:', rmse)
print('Coeficiente R2:', r2)
print('Variância Explicada:', evs)

MAE - Erro Médio Absoluto: 49.73811870313032
MSE - Erro Quadrático Médio: 4558.624461869952
RMSE - Raiz Quadrada do Erro Quadrático Médio: 67.5175863154923
Coeficiente R2: 0.9573789613443023
Variância Explicada: 0.9579785922314106


In [58]:
# Salvar os parâmetros das transformações e o modelo
joblib.dump(scaler_X, 'pipeline/scaler_X.pkl')
joblib.dump(scaler_y, 'pipeline/scaler_y.pkl')
joblib.dump(model, 'pipeline/model.pkl')
joblib.dump(winsor_limits, 'pipeline/winsor_limits.pkl')

print('tudo salvo.')

tudo salvo.


In [60]:
# Carregar os parâmetros salvos e o modelo para novos dados
scaler_X = joblib.load('pipeline/scaler_X.pkl')
scaler_y = joblib.load('pipeline/scaler_y.pkl')
model = joblib.load('pipeline/model.pkl')
winsor_limits = joblib.load('pipeline/winsor_limits.pkl')

# Exemplo de novos dados
new_data = pd.DataFrame({
    'feature1': [549.671415],
    'feature2': [1639.935544]
})

# Aplicar as mesmas transformações aos novos dados (Winsorização e Padronização)
new_data_winsor = apply_winsorization(new_data, winsor_limits)
new_data_scaled = scaler_X.transform(new_data_winsor)

# Fazer previsões nos novos dados
predictions_scaled = model.predict(new_data_scaled)

# Reverter a padronização das previsões para a escala original
predictions_original_scale = scaler_y.inverse_transform(predictions_scaled)

print("Previsões (em escala original):", predictions_original_scale)

Previsões (em escala original): [[4928.86152035]]


<br><br>

# Exemplo de Preparação de Dados e Criação de Modelo com Classificação Multiclasse

In [80]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.stats.mstats import winsorize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

# Exemplo de dados
np.random.seed(42)  # Para reprodutibilidade

# Gerar dados
N = 1000

# Dividir N em três partes iguais
N_per_class = N // 3

feature1_class1 = np.random.randn(N_per_class + 1) * 50 + 500
feature2_class1 = np.random.randn(N_per_class + 1) * 50 + 1500
feature1_class2 = np.random.randn(N_per_class) * 50 + 700
feature2_class2 = np.random.randn(N_per_class) * 50 + 1300
feature1_class3 = np.random.randn(N_per_class) * 50 + 900
feature2_class3 = np.random.randn(N_per_class) * 50 + 1100

feature1 = np.concatenate([feature1_class1, feature1_class2, feature1_class3])
feature2 = np.concatenate([feature2_class1, feature2_class2, feature2_class3])
feature3 = np.random.choice(['A', 'B', 'C'], size=N)

target = np.array(['Class1'] * N_per_class + ['Class2'] * N_per_class + ['Class3'] * N_per_class)
target = np.append(target, 'Class3')

# Verificar se todas as arrays têm o mesmo comprimento
print(len(feature1), len(feature2), len(feature3), len(target))

# Criar DataFrame
dados = pd.DataFrame({
    'feature1': feature1,
    'feature2': feature2,
    'feature3': feature3,
    'target': target
})

display(dados.head(2))


# Codificar variáveis categóricas
label_encoder_features = LabelEncoder()
dados['feature3'] = label_encoder_features.fit_transform(dados['feature3'])

# Codificar a variável alvo
label_encoder_target = LabelEncoder()
dados['target'] = label_encoder_target.fit_transform(dados['target'])

display(dados.head(2))

1000 1000 1000 1000


,feature1,feature2,feature3,target
0,524.835708,1548.755987,B,Class1
1,493.086785,1492.647131,C,Class1


,feature1,feature2,feature3,target
0,524.835708,1548.755987,1,0
1,493.086785,1492.647131,2,0


In [81]:
## Separar os dados em treino e teste

# Cria um objeto separado para a variável alvo
y = dados['target']

# Cria um objeto separado para as variáveis de entrada
X = dados.drop('target', axis=1)

# Split em dados de treino e teste sem amostragem estratificada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print do shape
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(800, 3) (200, 3) (800,) (200,)


In [82]:
## Tratando Valores Outliers

# Função para aplicar Winsorização
def apply_winsorization(X, limits):
    return X.apply(lambda col: winsorize(col, limits=limits))

winsor_limits = (0.05, 0.05)

# Winsorização das variáveis numéricas contínuas nos dados de treino e teste
X_train[['feature1', 'feature2']] = apply_winsorization(X_train[['feature1', 'feature2']], winsor_limits)
X_test[['feature1', 'feature2']] = apply_winsorization(X_test[['feature1', 'feature2']], winsor_limits)

In [83]:
## Padronização
scaler = StandardScaler()

# Padronização das variáveis numéricas contínuas nos dados de treino e teste
X_train[['feature1', 'feature2']] = scaler.fit_transform(X_train[['feature1', 'feature2']])
X_test[['feature1', 'feature2']] = scaler.transform(X_test[['feature1', 'feature2']])

In [84]:
## Treinar o Modelo

# Treinar o modelo de classificação
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


## Previsões

# Fazer previsões nos dados de teste
y_pred = model.predict(X_test)

# Calcular métricas de avaliação
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Acurácia: {accuracy:.4f}")
print(f"Precisão: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Acurácia: 0.9850
Precisão: 0.9857
Recall: 0.9850
F1-Score: 0.9851


In [85]:
# Salvar os parâmetros das transformações e o modelo
joblib.dump(scaler, 'pipeline/scaler.pkl')
joblib.dump(label_encoder_features, 'pipeline/label_encoder_features.pkl')
joblib.dump(label_encoder_target, 'pipeline/label_encoder_target.pkl')
joblib.dump(model, 'pipeline/model.pkl')
joblib.dump(winsor_limits, 'pipeline/winsor_limits.pkl')

print('tudo salvo.')

tudo salvo.


In [88]:
# Carregar os parâmetros salvos e o modelo para novos dados
scaler = joblib.load('pipeline/scaler.pkl')
label_encoder_features = joblib.load('pipeline/label_encoder_features.pkl')
label_encoder_target = joblib.load('pipeline/label_encoder_target.pkl')
model = joblib.load('pipeline/model.pkl')
winsor_limits = joblib.load('pipeline/winsor_limits.pkl')

# Exemplo de novos dados
new_data = pd.DataFrame({
    'feature1': [524.835708, 1500],
    'feature2': [1548.755987, 800],
    'feature3': ['B', 'C']
})

display(new_data)

# Codificar variáveis categóricas nos novos dados
new_data['feature3'] = label_encoder_features.transform(new_data['feature3'])

# Aplicar as mesmas transformações aos novos dados (Winsorização e Padronização)
new_data[['feature1', 'feature2']] = apply_winsorization(new_data[['feature1', 'feature2']], winsor_limits)
new_data[['feature1', 'feature2']] = scaler.transform(new_data[['feature1', 'feature2']])

# Fazer previsões nos novos dados
predictions = model.predict(new_data)

# Reverter a codificação das previsões para as classes originais
predictions_original = label_encoder_target.inverse_transform(predictions)

print("Previsões (classe original):", predictions_original)

,feature1,feature2,feature3
0,524.835708,1548.755987,B
1,1500.000000,800.000000,C


Previsões (classe original): ['Class1' 'Class3']


<br> <br> <br>

# O que é VectorAssembler no Apache Spark?

### VectorAssembler

- A função VectorAssembler no Apache Spark é um transformador que combina uma lista de colunas em uma única coluna de vetor. Este vetor de atributos é utilizado como entrada para treinar modelos de Machine Learning. O VectorAssembler é especialmente útil quando precisamos combinar várias características (features) de entrada em uma única representação vetorial, que é o formato esperado por muitos algoritmos de Machine Learning.

#### Tipos de Colunas Aceitas

O VectorAssembler aceita os seguintes tipos de colunas de entrada:

- Todos os tipos numéricos (inteiros, flutuantes, etc.)
- Tipo booleano
- Tipo vetorial (como DenseVector ou SparseVector)

#### Funcionamento

- Em cada linha do DataFrame, os valores das colunas de entrada especificadas são concatenados em um vetor na ordem que você define. Este vetor de características (features) será então usado como variável de entrada para treinar o modelo de Machine Learning.

<br>

### Este pipeline completo demonstra como incluir a padronização após a codificação das variáveis categóricas e a combinação das características em um vetor. 

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

# Inicializar SparkSession
spark = SparkSession.builder.appName("CategoricalEncoding").getOrCreate()

# Exemplo de dados
data = [
    (0, "a", "x", 10.0),
    (1, "b", "y", 20.0),
    (2, "c", "z", 30.0),
    (3, "a", "x", 40.0),
    (4, "a", "y", 50.0),
    (5, "c", "z", 60.0)
]

# Criar DataFrame
columns = ["id", "category1", "category2", "value"]
df = spark.createDataFrame(data, columns)

# Mostrar DataFrame
df.show()

# StringIndexer para category1 e category2
indexer1 = StringIndexer(inputCol="category1", outputCol="category1_index")
indexer2 = StringIndexer(inputCol="category2", outputCol="category2_index")

# OneHotEncoder para category1_index e category2_index
encoder1 = OneHotEncoder(inputCol="category1_index", outputCol="category1_encoded")
encoder2 = OneHotEncoder(inputCol="category2_index", outputCol="category2_encoded")

# VectorAssembler para combinar todas as colunas de recursos em uma única coluna de vetor
assembler = VectorAssembler(
    inputCols=["category1_encoded", "category2_encoded", "value"],
    outputCol="features"
)

# StandardScaler para padronizar as características
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

# Pipeline
pipeline = Pipeline(stages=[indexer1, indexer2, encoder1, encoder2, assembler, scaler])

# Ajustar e transformar os dados
pipeline_model = pipeline.fit(df)
df_transformed = pipeline_model.transform(df)

# Mostrar DataFrame transformado
df_transformed.select("id", "category1", "category2", "value", "scaled_features").show()

24/08/06 17:16:51 WARN Utils: Your hostname, eduardo-Inspiron-15-3520 resolves to a loopback address: 127.0.1.1; using 192.168.0.13 instead (on interface wlp0s20f3)
24/08/06 17:16:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/06 17:16:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/06 17:16:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

+---+---------+---------+-----+
| id|category1|category2|value|
+---+---------+---------+-----+
|  0|        a|        x| 10.0|
|  1|        b|        y| 20.0|
|  2|        c|        z| 30.0|
|  3|        a|        x| 40.0|
|  4|        a|        y| 50.0|
|  5|        c|        z| 60.0|
+---+---------+---------+-----+



+---+---------+---------+-----+--------------------+
| id|category1|category2|value|     scaled_features|
+---+---------+---------+-----+--------------------+
|  0|        a|        x| 10.0|[1.82574185835055...|
|  1|        b|        y| 20.0|(5,[3,4],[1.93649...|
|  2|        c|        z| 30.0|(5,[1,4],[1.93649...|
|  3|        a|        x| 40.0|[1.82574185835055...|
|  4|        a|        y| 50.0|[1.82574185835055...|
|  5|        c|        z| 60.0|(5,[1,4],[1.93649...|
+---+---------+---------+-----+--------------------+



----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 36406)
Traceback (most recent call last):
  File "/home/eduardo/anaconda3/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/eduardo/anaconda3/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/eduardo/anaconda3/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/eduardo/anaconda3/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/eduardo/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/eduardo/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/home/eduardo/anaconda3/

<br> <br>

# O que é Vetores Densos e Vetores Esparsos em Objetos RDD?

### Vetores Densos

#### Descrição:
- Vetores densos são vetores em que todos os elementos são armazenados explicitamente, independentemente de serem zero ou não. Eles são representados como uma lista ou um array que contém todos os valores dos elementos na mesma ordem em que aparecem.

#### Uso:
- Os vetores densos são mais eficientes e apropriados quando a maioria dos valores são diferentes de zero. Isso porque o armazenamento explícito de todos os elementos não desperdiça muito espaço, já que há poucos zeros. São ideais em situações onde os dados são completos e não esparsos.

#### Exemplo em RDD:

In [2]:
from pyspark.mllib.linalg import Vectors
from pyspark import SparkContext

# Inicializar SparkContext
sc = SparkContext.getOrCreate()

# Exemplo de dados
data = [
    (1, Vectors.dense([1.0, 2.0, 3.0])),
    (0, Vectors.dense([4.0, 5.0, 6.0])),
    (1, Vectors.dense([7.0, 8.0, 9.0]))
]

# Criar RDD
rdd = sc.parallelize(data)
print(rdd.collect())

[(1, DenseVector([1.0, 2.0, 3.0])), (0, DenseVector([4.0, 5.0, 6.0])), (1, DenseVector([7.0, 8.0, 9.0]))]


<br>

### Vetores Esparsos

#### Descrição:
- Vetores esparsos são vetores em que apenas os elementos não-zero são armazenados, junto com suas posições. Eles são representados de forma compacta, especificando o tamanho do vetor, uma lista de índices de elementos não-zero, e os valores correspondentes a esses índices.

### Uso:
- Os vetores esparsos são ideais quando a maioria dos valores no vetor são zero, economizando espaço e tempo de computação. Eles são especialmente úteis em problemas de Machine Learning e Data Mining, onde os dados são frequentemente esparsos, como em representações de texto (bag-of-words) e matrizes de incidência.

#### Exemplo em RDD:

In [3]:
from pyspark.mllib.linalg import Vectors
from pyspark import SparkContext

# Inicializar SparkContext
sc = SparkContext.getOrCreate()

# Exemplo de dados
data = [
    (1, Vectors.sparse(5, {0: 1.0, 3: 2.0})),
    (0, Vectors.sparse(5, {1: 3.0, 4: 4.0})),
    (1, Vectors.sparse(5, {2: 5.0, 3: 6.0}))
]

# Criar RDD
rdd = sc.parallelize(data)
print(rdd.collect())

[(1, SparseVector(5, {0: 1.0, 3: 2.0})), (0, SparseVector(5, {1: 3.0, 4: 4.0})), (1, SparseVector(5, {2: 5.0, 3: 6.0}))]


24/08/06 19:45:09 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2358952 ms exceeds timeout 120000 ms
24/08/06 19:45:09 WARN SparkContext: Killing executors is not supported by current scheduler.


### Resumo

- **Vetores Densos**: Utilize-os em RDDs quando a maioria dos dados são não-zero, como em representações de características contínuas onde poucas entradas são zero.
- **Vetores Esparsos**: Utilize-os em RDDs quando a maioria dos dados são zeros, como em representações textuais ou dados de alta dimensionalidade onde apenas algumas dimensões têm valores não-zero.

<br><br><br><br>

# Pré-Processamento de Dados Para Construção de Modelos de Machine Learning

<br><br>

## Redução de Dimensionalidade com PCA

A redução de dimensionalidade deve ser aplicada quando o número de vaiáveis preditoras for muito alto.

A redução de dimensionalidade é uma técnica usada para reduzir o número de variáveis preditoras em um conjunto de dados. Isso é útil quando se trabalha com um grande número de variáveis, pois pode simplificar o modelo, reduzir o tempo de processamento e ajudar a evitar o overfitting (quando o modelo se ajusta demais aos dados de treinamento e não generaliza bem para novos dados).

<br>

In [ ]:
# Cria o objeto PCA com 3 componentes
bankPCA = PCA(k = 3, inputCol = "features", outputCol = "pcaFeatures")

# Treina o modelo
pcaModel = bankPCA.fit(bankDF)

# Aplica o modelo PCA para reduzir a dimensionalidade
pcaResult = pcaModel.transform(bankDF).select("label", "pcaFeatures")

# Visualiza o Resultado
pcaResult.show(truncate = False)